# Simple, Phase 1, approximate flight profile simulation

In [39]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import control

# Purdue AA451 library
from scripts.jsbsim_utils import Logger, trim, simulate, linearize, rootlocus, clean_tf

In [40]:
aircraft_model = 'WaveRipper'

# Ground Trim

In [41]:
op_ground, props = trim(
    aircraft=aircraft_model,
    ic={
        'ic/vt-fps': 0,
        'ic/h-agl-ft': 5,
        'ic/psi-true-deg': 280,
        'fcs/left-brake-cmd-norm': 1,
        'fcs/right-brake-cmd-norm': 1,
        'fcs/center-brake-cmd-norm': 1,
    },
    design_vector=['ic/theta-rad', 'ic/h-agl-ft'],
    x0=[0, 0],
    verbose=True,
    method='Nelder-Mead', # works better with ground interaction
    ftol=1e-2,
)
op_ground

 final_simplex: (array([[-0.00398416, -0.0198497 ],
       [-0.00397723, -0.01982051],
       [-0.00397813, -0.01978912]]), array([0.0010305 , 0.00103479, 0.00109463]))
           fun: 0.0010305007323130465
       message: 'Optimization terminated successfully.'
          nfev: 78
           nit: 41
        status: 0
       success: True
             x: array([-0.00398416, -0.0198497 ])



  The aerodynamic axis system has been set by default to the Lift/Side/Drag system.

  The aerodynamic moment axis system has been set by default to the bodyXYZ system.


{'ic/vt-fps': 0,
 'ic/h-agl-ft': -0.01984969996708967,
 'ic/psi-true-deg': 280,
 'fcs/left-brake-cmd-norm': 1,
 'fcs/right-brake-cmd-norm': 1,
 'fcs/center-brake-cmd-norm': 1,
 'ic/theta-rad': -0.0039841619774816615}

In [42]:
op_ground

{'ic/vt-fps': 0,
 'ic/h-agl-ft': -0.01984969996708967,
 'ic/psi-true-deg': 280,
 'fcs/left-brake-cmd-norm': 1,
 'fcs/right-brake-cmd-norm': 1,
 'fcs/center-brake-cmd-norm': 1,
 'ic/theta-rad': -0.0039841619774816615}

In [43]:
log_ground = simulate(
    aircraft=aircraft_model,
    op_0=op_ground,
    tf=5,
    realtime=False, verbose=True)


  The aerodynamic axis system has been set by default to the Lift/Side/Drag system.

  The aerodynamic moment axis system has been set by default to the bodyXYZ system.


## Hover Trim

In [49]:
op_hover, props = trim(
    aircraft=aircraft_model,
    ic={
        'ic/h-sl-ft': 650,      # Height above sea level initial condition in feet
        'ic/vt-fps': 500,       # True airspeed initial condition in feet/second
        'ic/psi-true-deg': 280, # Heading angle initial condition in degrees
    },
    eq_constraints = [
        lambda fdm: fdm['accelerations/udot-ft_sec2'],
        #lambda fdm: fdm['accelerations/vdot-ft_sec2'],
        lambda fdm: fdm['accelerations/wdot-ft_sec2'],
        #lambda fdm: fdm['accelerations/pdot-rad_sec2'],
    ],
    design_vector=[
        'fcs/throttle-cmd-norm',
        'fcs/elevator-cmd-norm',
    ],
    x0=[0.9, 0.5],
    cost= lambda fdm: fdm['fcs/throttle-cmd-norm'],
    verbose=True,
    method='SLSQP',
    bounds=[[0, 1], [-1, 1]],
)
op_hover

     fun: 0.8999999999999997
     jac: array([1., 0.])
 message: 'Positive directional derivative for linesearch'
    nfev: 347
     nit: 53
    njev: 49
  status: 8
 success: False
       x: array([ 0.9       , -0.79099763])
constraint eq 42.76133080578421
constraint eq 32.21183905857108



  The aerodynamic axis system has been set by default to the Lift/Side/Drag system.

  The aerodynamic moment axis system has been set by default to the bodyXYZ system.


RuntimeError: trim failed:
     fun: 0.8999999999999997
     jac: array([1., 0.])
 message: 'Positive directional derivative for linesearch'
    nfev: 347
     nit: 53
    njev: 49
  status: 8
 success: False
       x: array([ 0.9       , -0.79099763])
{'accelerations/a-pilot-x-ft_sec2': 42.76133035077288, 'accelerations/a-pilot-y-ft_sec2': 3.6409805536435045e-11, 'accelerations/a-pilot-z-ft_sec2': -4.1359030627651384e-25, 'accelerations/n-pilot-x-norm': 1.3290627779022985, 'accelerations/n-pilot-y-norm': 1.1316513516343913e-12, 'accelerations/n-pilot-z-norm': -1.2854779700823567e-26, 'accelerations/Nx': 1.329062778275642, 'accelerations/Ny': 0.0, 'accelerations/Nz': -0.0, 'accelerations/pdot-rad_sec2': 2.5848337571834437e-09, 'accelerations/qdot-rad_sec2': -4.557759320374709e-10, 'accelerations/rdot-rad_sec2': -5.113571937046131e-10, 'accelerations/udot-ft_sec2': 42.76133080578421, 'accelerations/vdot-ft_sec2': 0.047274842150382745, 'accelerations/wdot-ft_sec2': 32.21183905857108, 'ac

In [ ]:
log_hover = simulate(
    aircraft=aircraft_model,
    op_0=op_hover,
    tf=10,
    realtime=False)

In [ ]:
op_hover

In [ ]:
op_hover_auto = dict(op_hover)
op_hover_auto['ic/theta-deg'] = 0
op_hover_auto['ic/phi-deg'] = 0
op_hover_auto['ic/h-agl-ft'] = 60

op_hover_auto['ap/heading-cmd-deg'] = 280
op_hover_auto['ap/gear-enable'] = 1
op_hover_auto['ap/roll-enable'] = 0
op_hover_auto['ap/pitch-enable'] = 1
op_hover_auto['ap/yaw-enable'] = 0
op_hover_auto['ap/h-enable'] = 1
op_hover_auto['ap/h-sl-cmd-ft'] = 700

log_hover_auto = simulate(
    aircraft='F-35B-2',
    op_0=op_hover_auto,
    tf=50,
    realtime=False, verbose=True)

In [ ]:
log_hover_auto['fcs/throttle-pos-norm'].plot(label='0')
log_hover_auto['fcs/throttle-pos-norm[1]'].plot(label='1')
log_hover_auto['fcs/throttle-pos-norm[2]'].plot(label='2')
log_hover_auto['fcs/throttle-pos-norm[3]'].plot(label='3')

plt.legend()

In [ ]:
log_hover_auto['ap/h-error'].plot()

## Transition

## Cruise Trim

In [ ]:
op_cruise, props = trim(
    aircraft=aircraft_model,
    ic={
        'ic/gamma-deg': 0,
        'ic/vt-fps': 600,
        'ic/h-sl-ft': 10000,
        'gear/gear-cmd-norm': 0,
        'fcs/left-brake-cmd-norm': 0,
        'fcs/right-brake-cmd-norm': 0,
        'fcs/center-brake-cmd-norm': 0,
        'propulsion/engine/pitch-angle-rad': 0,
    },
    design_vector=[
        'fcs/throttle-cmd-norm',
        'fcs/elevator-cmd-norm',
        'fcs/rudder-cmd-norm',
        'fcs/aileron-cmd-norm',
        'ic/alpha-rad',
        'ic/beta-rad',
    ],
    method='SLSQP',
    eq_constraints= [
        lambda fdm: fdm['accelerations/udot-ft_sec2'],
        lambda fdm: fdm['accelerations/vdot-ft_sec2'],
        lambda fdm: fdm['accelerations/wdot-ft_sec2'],
        lambda fdm: fdm['accelerations/pdot-rad_sec2'],
        lambda fdm: fdm['accelerations/qdot-rad_sec2'],
        lambda fdm: fdm['accelerations/rdot-rad_sec2'],
    ],
    cost=lambda fdm: fdm['fcs/throttle-cmd-norm'],
    x0=[0.5, 0, 0, 0, 0, 0],
    verbose=False,
    bounds=[[0, 1], [-1, 1], [-1, 1], [-1, 1], [-1, 1], [-1, 1]],
    tol=1e-3
)
op_cruise

In [ ]:
log_cruise = simulate(
    aircraft='F-35B-2',
    op_0=op_cruise,
    tf=10,
    realtime=False)

In [ ]:
op_cruise_auto = dict(op_cruise)
#op_cruise_auto['ic/theta-deg'] = 0
#op_cruise_auto['ic/phi-deg'] = 0
op_cruise_auto['ap/roll-enable'] = 1
op_cruise_auto['ap/pitch-enable'] = 1
op_cruise_auto['ap/yaw-enable'] = 1
op_cruise_auto['ap/h-enable'] = 1
op_cruise_auto['ap/heading-cmd-deg'] = 260
op_cruise_auto['ap/h-sl-cmd-ft'] = 10000

log_cruise_auto = simulate(
    aircraft='F-35B-2',
    op_0=op_cruise_auto,
    tf=40,
    realtime=False)